In [1]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.01

class A_2(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_2(GenModel):
    def get_agent_config(self):
        return A_2
    
    def init_env(self):
        #no influence
        return [self.env_val] * 5000
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M2'
        
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        
        
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_LOW = settings.INIT_MEAN - 1 * 2 * settings.INIT_SD
        settings.INIT_HIGH = settings.INIT_MEAN + 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (self.env_val, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step',
             TYPE_MAP[0], TYPE_MAP[0] + '_beta', TYPE_MAP[0] + '_gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + '_beta', TYPE_MAP[1] + '_gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        
        data = self.collect_data()
        #print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        if self.schedule.steps % 10 == 0: 
            self.data_file.flush()
        super().step()
        
        


In [ ]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm2 1 0.01'.split(' ')

env_val, muta_sd = float(sys.argv[1]), float(sys.argv[2])

fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'env_val': env_val , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':muta_sd, 'muta_sd':muta_sd}]}

batch_run = BatchRunnerMP(
    M_2, 2,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=2,
    max_steps=2000,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./res/M2/1.0_0.01_0.01/7ff967ee-ced5-11e9-b3df-f2189850a77c
data in file  ./res/M2/1.0_0.01_0.01/7ff9c9fa-ced5-11e9-8ca4-f2189850a77c
0 population size:  2000
0 population size:  2000
1 population size:  2651
1 population size:  2666
2 population size:  3395
2 population size:  3395
3 population size:  4071
3 population size:  4116
4 population size:  4767
4 population size:  4745
5 population size:  5355
5 population size:  5361


In [4]:
np.clip(np.random.normal(loc=settings.GENE_INIT_MEAN, scale=settings.GENE_INIT_SD, size=10), settings.GENE_INIT_LOW, settings.GENE_INIT_HIGH)

array([-0.4, -0.4, -0.4, -0.4, -0.4,  0.4, -0.4, -0.4, -0.4, -0.4])

In [1]:
a = [0.7505020268345521, 0.7511713736642536, 0.7491957416873626, 0.7521043323976395, 0.7479876951136514, 0.7497239873962237, 0.7470013503961077, 0.7510766424880152, 0.7515421551842142, 0.7524874800805714, 0.7506625829845465, 0.7509540251441067, 0.7513844827913161, 0.7495417767045023, 0.7496625836037214, 0.7488408418598628, 0.7496034033119632, 0.7505450834230494, 0.749940459915391, 0.7504595222738819, 0.7484249104182865, 0.7499000906883133, 0.7487463812835209, 0.7487981181230615, 0.7465992221752782, 0.7504218277350228, 0.7479546134980313, 0.7484536277285555, 0.751376574622242, 0.7503104113373908, 0.7492262892203896, 0.7500021591719727, 0.750029679151594, 0.7488899746441124, 0.7502394262416083, 0.7500041949412994, 0.7497111146434158, 0.750287358935235, 0.7496860677815528, 0.7477928481539877, 0.7503875772825839, 0.7487960213155709, 0.7492457724439248, 0.7518878373707416, 0.7529831271118832, 0.750585679797385, 0.7494158517001783, 0.7503588806268013, 0.747622170087284, 0.7513258924271506, 0.7481289885107458, 0.748256525447994, 0.7502128347046652, 0.7505896954640698, 0.750322440714503, 0.7489824629986244, 0.7487652778216183, 0.7490923930818318, 0.74776870525128, 0.7515977867508925, 0.7508568493155283, 0.7493534609364563, 0.7502827814465789, 0.7499384668214469, 0.7489783673034465, 0.7476738296461991, 0.7494880056140268, 0.7514191997544719, 0.7510914513563776, 0.7534170255729054, 0.750994912500022, 0.74972273688549, 0.750295944720078, 0.7506116290379521, 0.7497810561803945, 0.7496897565867111, 0.7522851131023942, 0.7517126684825447, 0.7510491056053561, 0.7508275911398832, 0.7489702766341745, 0.7495988895403375, 0.7500658599980851, 0.7487369875207398, 0.7503177285031472, 0.7484777204949038, 0.7497323217516254, 0.750034279407727, 0.7472134434129974, 0.7495616230341583, 0.7522444863815942, 0.7494472723795343, 0.7502385259797191, 0.7503594599541901, 0.7505515111208758, 0.7490563314528818, 0.7510952559077324, 0.7502791506790178, 0.7499720463830577, 0.7504879764267508, 0.7517045624444274, 0.7522684829297346, 0.7495150924703691, 0.7517041984629332, 0.7491622025887241, 0.7479813413680445, 0.7515514360200934, 0.7523233010494609, 0.7487356158283058, 0.7514987768145454, 0.7501104321394213, 0.7539983882846593, 0.7503466544296611, 0.7501146693625156, 0.7535106438854173, 0.7495537275601926, 0.7492266401750962, 0.749740375866204, 0.7503435713928008, 0.7502979696530458, 0.7502432228156564, 0.7484067102720924, 0.7495121746865865, 0.7480712144848529, 0.7486061333218795, 0.7490225097928023, 0.7464282793169689, 0.7483052626647754, 0.7487559608609687, 0.7502070984999429, 0.751407419019676, 0.7494801952822453, 0.750657850214822, 0.7507267061550535, 0.7531674360524482, 0.7490864782352675, 0.7469435479126215, 0.7486788076694861, 0.7519877580303845, 0.7520323983396429, 0.7490295221853246, 0.7518159135253257, 0.7506000921347429, 0.7500772083676308, 0.7519063749106766, 0.7507295925867956, 0.749724166114011, 0.7528318163586785, 0.7522358596220491, 0.7512817200173367, 0.7506659036697854, 0.7496708068010567, 0.7483609323216316, 0.7476106252541335, 0.7497420481936626, 0.7497243368799931, 0.7511852797024329, 0.7480057228114552, 0.7508492187032545, 0.7474709627672512, 0.751319467078748, 0.7512918030441365, 0.7496097023414052, 0.7506623216023635, 0.7498613588689761, 0.750582301723988, 0.7489774675107002, 0.7512930544308262, 0.7526474146902964, 0.7488275602392717, 0.750371033084479, 0.7498923018644471, 0.7521806746209487, 0.7513229972450501, 0.7510212937058576, 0.7500561860605217, 0.7517034603392736, 0.7507476093979324, 0.7510885447646796, 0.7517661090524185, 0.7504429497847858, 0.7499882571814963, 0.750553930200627, 0.7499667993550101, 0.7497978064924932, 0.7515077449421956, 0.7491701114576944, 0.7508320075321016, 0.7486061598734416, 0.7519560709234978, 0.7502260775649874, 0.7503875736466081, 0.7488620853408094, 0.7500550100063339, 0.7515780843824974, 0.7508197547555828, 0.7492220073289151, 0.7486086577114481, 0.7506722636165453, 0.7516319954208122, 0.7490344925714134, 0.7485610561134861, 0.7491280435433948, 0.7490941899715762, 0.7487199576920767, 0.7509486891025127, 0.7511170731817886, 0.7501399285298227, 0.7491536068105094, 0.7513810160841246, 0.749341104417833, 0.7498223337695658, 0.7490402640181357, 0.7482079051047882, 0.7496551762984801, 0.7480495024654277, 0.7498503162472131, 0.7501813806072171, 0.7512013084347391, 0.7505372747674899, 0.7495715809265303, 0.7499913058233988, 0.7500092830184277, 0.7516669952124242, 0.7492870524057825, 0.7484314971834849, 0.7513664015713108, 0.7475377656940386, 0.7507566420417326, 0.7499587323529299, 0.7478506543055499, 0.7493837188441935, 0.7516684979128248, 0.7490988394553086, 0.7481520108147977, 0.7504691659907218, 0.7494562364622952, 0.75111335350226, 0.752438130251009, 0.7525383817887742, 0.7467127671794569, 0.7487350570452797, 0.7498745233307091, 0.7500908561007518, 0.7501334822718888, 0.749570240094482, 0.7497670785890787, 0.7506576240999285, 0.7508501009013124, 0.7517583081401007, 0.7518585973577481, 0.7507272077595637, 0.7518367414633617, 0.7524191964118248, 0.7515917797221856, 0.7500469944589477, 0.750031584638319, 0.7511467022834128, 0.7494159120245287, 0.7488961943311198, 0.7506532818873988, 0.7484770672010379, 0.7519450921250167, 0.7505092182170179, 0.7498895564200831, 0.7497022032453854, 0.7522659071305611, 0.749933840263398, 0.751708236393283, 0.75257641335825, 0.7470021448211661, 0.7487326910167198, 0.7497682034973933, 0.749284293752602, 0.750613798590465, 0.7498368911188249, 0.7500412827866669, 0.7500485566150048, 0.7518113301829878, 0.7515303218813498, 0.7488798897488188, 0.7499943177606345, 0.7469880681749634, 0.7498262708184353, 0.7503222065065707, 0.7496966436946773, 0.7492508518012533, 0.7514131114267837, 0.7515547719095644, 0.7483201198967814, 0.7513751235807276, 0.7488632998501161, 0.7492306027146327, 0.7510760747709837, 0.7479603337088933, 0.7512055108154514, 0.7497497383635102, 0.7499762361251412, 0.7483091214050956, 0.7503017223024033, 0.7523776138596019, 0.7488784272995985, 0.7489417557269782, 0.7502268977327297, 0.7478280578920119, 0.7490340270681003, 0.7479968328521374, 0.7512947391816003, 0.7493893299234723, 0.7500911002521012, 0.7492184591083063, 0.7498744083590174, 0.7502510962855746, 0.7490093816020317, 0.7493647697349073, 0.748553814327135, 0.7533861896890535, 0.7485889428724494, 0.7475987694709254, 0.7519758338198219, 0.7474131483752635, 0.7515966552586562, 0.7492233575284842, 0.7526586952336071, 0.7483639848077168, 0.7486381999013696, 0.752300107482063, 0.7523275267446283, 0.7484172670775173, 0.7503471013399645, 0.7499086284745512, 0.750494902691685, 0.75148590135049, 0.751102441504055, 0.7487643188467343, 0.7517032091567157, 0.7491832807745511, 0.7491525598125796, 0.7520734992808309, 0.7495492526583702, 0.7505274452498494, 0.7505217898784857, 0.7494126265644852, 0.7478763489283748, 0.7495112908143831, 0.7487295382077147, 0.7507040917208332, 0.7497614633178706, 0.7500092628017232, 0.7499127272854975, 0.7516351515518703, 0.750013920809085, 0.7486103496828456, 0.7502179128010884, 0.7535540201847926, 0.7495600126073368, 0.7487840449155787, 0.7496474279253528, 0.7485628035718966, 0.7502233224151551, 0.7515188057493829, 0.7486654995997342, 0.7517899445212779, 0.7493586729361651, 0.7486278778046398, 0.7489295203871227, 0.7477945377474149, 0.7490434544852189, 0.7488007377363437, 0.7499912058862441, 0.7503108570197496, 0.748939838043049, 0.7503580985533504, 0.7514750878725545, 0.7481770841081974, 0.7508759889802448, 0.749481448581739, 0.7494449348285769, 0.748792521663898, 0.7504024003362068, 0.7484865302882922, 0.7503240535468843, 0.7496225925682753, 0.7509702370129088, 0.7498265589385386, 0.7504589799295963, 0.7492486013726666, 0.7502299064166449, 0.7510654676503098, 0.7499624366772162, 0.7508265854492935, 0.7494528430787035, 0.7514798576975686, 0.7502453635511449, 0.7497069615197752, 0.7492137701682038, 0.7518526785669578, 0.7493495385576948, 0.7478429803371744, 0.7502575650620233, 0.7522254395943783, 0.7524665140810117, 0.751658914779995, 0.7514939677661013, 0.748955417793393, 0.7491293781221973, 0.7481274385869027, 0.7472039955717766, 0.7515294580110033, 0.7515116941025465, 0.7504144307494904, 0.7491393940867064, 0.749114841817754, 0.7512798367635924, 0.7479128532330583, 0.7506979644059741, 0.7476862588528558, 0.7539307124986645, 0.7499016992028528, 0.7468710917464865, 0.7495277220595733, 0.7495798290345177, 0.7523395276315212, 0.7501571101623534, 0.7515717728382563, 0.7504150972909619, 0.7493981388349934, 0.7502367294168472, 0.748026696915571, 0.7526447237834288, 0.7483757256420036, 0.7514043186932156, 0.7481690722322278, 0.7509965718079543, 0.7507801676410266, 0.7493874264467844, 0.7498603316662323, 0.7493275375446034, 0.7508753101189449, 0.7507913194930307, 0.7518011099139519, 0.7496973604462005, 0.7495871884965812, 0.7503322923185293, 0.7504385261846478, 0.74904100274527, 0.7505275718102675, 0.7521107162581233, 0.7508708210560281, 0.7495941068807692, 0.7500431073253828, 0.751097828731321, 0.7488028761655992, 0.7509680913568961, 0.7475097617859915, 0.7501639029590726, 0.7518998653019329, 0.7488563463575844, 0.7504007884758048, 0.748691429496385, 0.7499334177438283, 0.7504546957989707, 0.7504043971494434, 0.7488963257151174, 0.7488129984457441, 0.74856094706678, 0.748649483838439, 0.7495064094362199, 0.7492564428729838, 0.7507398045726922, 0.7486429849296207, 0.7510447759282936, 0.7497593323828895, 0.7488426848577898, 0.7483192205634328, 0.7496623547760701, 0.748597199832188, 0.7512867159115366, 0.7490167621258027, 0.7505111693602085, 0.7482105667685267, 0.750744493962291, 0.7498153299691299, 0.7488248139343863, 0.7500093671007457, 0.7505907859172465, 0.7499849473284148, 0.7511515573614898, 0.7505188548990426, 0.7485593024020346, 0.7498434930356923, 0.7505730617186581, 0.7507122403300787, 0.7472802004548854, 0.7517688380948142, 0.7476582171401487, 0.7522148893931773, 0.7524561411026034, 0.7460679059973003, 0.7499164886391396, 0.748439208165814, 0.7500438627665801, 0.7489223330585666, 0.7490579092037358, 0.747819833193957, 0.7498687736334716, 0.7517344087139202, 0.7492592234895271, 0.7496760483876587, 0.7486554274919559, 0.7503765646830602, 0.7490265756929521, 0.7513608325500072, 0.7508756431917071, 0.7508206548638903, 0.7514273875030335, 0.7534122917120765, 0.7485812547244074, 0.7496476124878126, 0.7504207682548664, 0.748005914346366, 0.7523426322881615, 0.7503870111080564, 0.7490159528335726, 0.7522936208811126, 0.7523165585021362, 0.7505145484498523, 0.7488950705520359, 0.7489607574428906, 0.749872455993039, 0.7502243102290734, 0.751743523811099, 0.7499082576019723, 0.7500408228246656, 0.7520994769333786, 0.7488074770034903, 0.7519100713762761, 0.7495243377236767, 0.7485966093820544, 0.7495228940125975, 0.751386003253858, 0.7472066277007468, 0.751435848818501, 0.7499015441332945, 0.7500948470086088, 0.7503613922249202, 0.7500946483134793, 0.748863598692971, 0.7486173316992372, 0.7471751054263669, 0.7493185355475062, 0.749268875718114, 0.7504398546711407, 0.7512104410918727, 0.7506557849785698, 0.7489388633400365, 0.7501487385057948, 0.7512785287617423, 0.7522184571767131, 0.7470343471122642, 0.7499273648608128, 0.7494233513854514, 0.7515180126768024, 0.7502925201321657, 0.7496551349609031, 0.7502363202211424, 0.7491207296461091, 0.750365409255066, 0.7492771233827524, 0.7482351917320726, 0.7496816502707145, 0.7502135074822571, 0.7502960136262196, 0.7494624958290205, 0.7493593819709585, 0.7495526163384328, 0.7501949823836799, 0.7484493948270965, 0.751771916055282, 0.7499740125801444, 0.7491295657753522, 0.7475546273992403, 0.7510125947485793, 0.7501428763530262, 0.7526822997800708, 0.7472015178582195, 0.7496095537607916, 0.7486321843677199, 0.7516866861613087, 0.7501036522159403, 0.748958806355394, 0.7508301723071791, 0.7508146054240594, 0.7496984713010284, 0.7519208322120903, 0.7494515541875932, 0.7537025834668465, 0.7487249388755152, 0.7497106723558223, 0.7502498682274259, 0.7514970688186419, 0.7497831749840875, 0.7495189716462993, 0.750770937167401, 0.7500318493002617, 0.7489782223139614, 0.7493364443516884, 0.7500401284489756, 0.750136717215808, 0.7520279293296572, 0.7522111873618251, 0.7517286279450444, 0.7487754249101419, 0.748282714400283, 0.7494360878893783, 0.7498878883361662, 0.7499663942464269, 0.7487816144540235, 0.7485933040850427, 0.750989796148555, 0.7501650617647253, 0.7482279262864131, 0.7502543090417407, 0.7493374965054241, 0.749800629495214, 0.7495838491264923, 0.7508240130543218, 0.7492917857501642, 0.7502826281839647, 0.7489800225794921, 0.7509221476384487, 0.7492725809095064, 0.7476549696206316, 0.7517560982062623, 0.7498916705014855, 0.7487832065859283, 0.7482512870253081, 0.7496172889543087, 0.7509309472508803, 0.7502318624853046, 0.7512377111805291, 0.7485091556665092, 0.7476628819259938, 0.7481214297539758, 0.7494385618342387, 0.7496730182635485, 0.750828072679204, 0.7488830762301237, 0.7492051740844126, 0.7497134886862824, 0.7516300704245074, 0.7492748408496079, 0.7478291303364247, 0.7478222733999812, 0.7520285874408421, 0.7490658824192403, 0.7511009426860213, 0.7495386528495587, 0.7491137896948338, 0.7497687062321703, 0.7494081270710027, 0.7504293835902602, 0.7507084406196896, 0.7487843898682954, 0.7501890270634841, 0.7491941116410844, 0.7497210306962967, 0.7524390618618004, 0.7485371342668936, 0.7478084939288282, 0.749526785824438, 0.7484752091746367, 0.7486010562407764, 0.7497718288911426, 0.7511149023632997, 0.7491467872264254, 0.7510180883780566, 0.7478851452142165, 0.7501746519621245, 0.7480512599922351, 0.7507135473612145, 0.7504447499754704, 0.7511924106175735, 0.7486816724349566, 0.7516788095885135, 0.7518601343704088, 0.7517288291522954, 0.7508175962745091, 0.7495356137610759, 0.7510779731574603, 0.7501880138717969, 0.7500619012993005, 0.7489195504309739, 0.750825578296138, 0.7509648437405227, 0.751535142852112, 0.7513256355328669, 0.7514860363040425, 0.7508658067326888, 0.7496158087432659, 0.7488312910144497, 0.7513006422549777, 0.7514924750052023, 0.7518743957297596, 0.7501243082787215, 0.7485087329472397, 0.7486603046024791, 0.7490759851399289, 0.7530765600087592, 0.7513811050462369, 0.7511004069004628, 0.7507712027600579, 0.7468264477754166, 0.749229522198936, 0.751418379470554, 0.7489660772907705, 0.7503713867254348, 0.7506737657511448, 0.751458334035441, 0.7511235340509207, 0.7509467035993762, 0.7498443988845047, 0.7508035396992083, 0.7504691140873699, 0.7503034182249139, 0.7488232918562265, 0.74914687680612, 0.7515778824489627, 0.7500626246765965, 0.7485916675671687, 0.7523568594907319, 0.7514173696699502, 0.7517609092560005, 0.7490014339360198, 0.7495929854605063, 0.7510937568896875, 0.7503909054702813, 0.751258031966218, 0.7521378968616386, 0.7512894593448884, 0.7502881120619382, 0.7510015712671243, 0.7511298007133056, 0.7508230876949205, 0.7514335933227227, 0.7494098896271765, 0.7485549067117172, 0.7504982216533689, 0.7513287069981125, 0.7492001566083613, 0.7487606453639587, 0.7481938809990232, 0.751073040917356, 0.7495601081204558, 0.7492729477592894, 0.7518450922746087, 0.7492816228892332, 0.7503078275742048, 0.7514392877188237, 0.7501612494971193, 0.7504675770392988, 0.7498421260780138, 0.7493036134657075, 0.7498248749493543, 0.7488599852408496, 0.7498827009518201, 0.7497537154366044, 0.751087990335783, 0.7500492344555573, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(a)

1356